<a href="https://colab.research.google.com/github/PremWoods/PremWoods.github.io/blob/main/Project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#!pip install world-bank-data
import pandas as pd
import world_bank_data as wb
import scipy.cluster.hierarchy as sch

df = pd.DataFrame(wb.get_series('DC.ODA.TOTL.CD'))

df.to_csv("Aid_given.csv")

df2 = pd.read_csv("Aid_given.csv")

df3 = df2.drop("Series", axis=1)

df4 = df3.dropna(axis=0)

df4['DC.ODA.TOTL.CD'] = df4['DC.ODA.TOTL.CD']/1000000

df4.rename(columns={'DC.ODA.TOTL.CD': 'Aid_mil'}, inplace=True)

df5 = df4.set_index("Country")

df5_wide = df5.pivot_table(values='Aid_mil', index='Year', columns='Country')

df5_wide.to_csv('Time_aid_donors.csv')

df3.to_csv('Aid_given.csv')

<ipython-input-6-b826dfa5bb36>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['DC.ODA.TOTL.CD'] = df4['DC.ODA.TOTL.CD']/1000000
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
import pandas as pd
import world_bank_data as wb
import scipy.cluster.hierarchy as sch

df1 = pd.DataFrame(wb.get_series('NY.GNP.MKTP.CD'))
df1.to_csv("GNI.csv")



In [ ]:
import numpy as np

df1 = pd.read_csv("GNI.csv")
df2 = pd.read_csv('Aid_given.csv')

df1[['Year']] = df1[['Year']].astype(str)
df2[['Year']] = df2[['Year']].astype(str)

result = pd.merge(df1, df2, on=['Country', 'Year'])
result2 = result.dropna()
result3 = result2.drop(["Series", "Unnamed: 0"], axis=1)
result4 = result3.set_index("Country")
result4['C'] = result4['DC.ODA.TOTL.CD'] / result4['NY.GNP.MKTP.CD']
result4['GNI_perc'] = result4['C']*100
result4['ODA_mil'] = result4['DC.ODA.TOTL.CD']/1000000
result4['GNI_bil'] = result4['NY.GNP.MKTP.CD']/1000000000
result5 = result4.drop(['NY.GNP.MKTP.CD','DC.ODA.TOTL.CD', 'C'], axis =1)
result6 = result5[result5['Year'] == "2020"].reset_index(drop=False)
result6['UN'] = np.where(result6['GNI_perc'] < 0.7, 0, 1)
result6.to_csv("2020_aid.csv")

In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv("GNI.csv")
df2 = pd.read_csv("ODA_2021.csv")
key = pd.read_json("https://raw.githubusercontent.com/PremWoods/PremWoods.github.io/main/KeyCountry.json")

df1[['Year']] = df1[['Year']].astype(str)
df2[['Year']] = df2[['TIME']].astype(str)

df3 = df2.drop(["INDICATOR", "SUBJECT","FREQUENCY", "TIME","Flag Codes"], axis=1)
df3['Location'] = df3['LOCATION'].apply(lambda x: x.lower())
df4 = pd.merge(df3, key, left_on='Location', right_on='alpha3')

df5 = df4.drop(["MEASURE", "LOCATION","id", "alpha2","Location","alpha3"], axis=1)
df5.rename(columns={'name': 'Country','Value':'ODA_mil'}, inplace=True)


df5[['Year']] = df5[['Year']].astype(str)
df1[['Year']] = df1[['Year']].astype(str)

result = pd.merge(df5, df1, on=['Country', 'Year'])

result1 = result[result['Year'] == "2021"].reset_index(drop=True)
result2 = result1.dropna()
result3 = result2.drop("Series", axis=1)
result4 = result3.set_index("Country")

result4['C'] = result4['NY.GNP.MKTP.CD']/1000000
result4['GNI_bil'] = result4['NY.GNP.MKTP.CD']/1000000000
result4['D'] = result4['ODA_mil']/ result4['C']
result4['GNI_perc'] = result4['D']*1000
result5 = result4.drop(['NY.GNP.MKTP.CD','D', 'C'], axis =1)
result5['UN'] = np.where(result5['GNI_perc'] < 0.7, 0, 1)
result5.to_csv('2021_aid.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('SSA_UKR_AIDCOMP.csv')
df2 = df.set_index("Country Name").stack().reset_index()

df3 = df2.pivot_table(index='level_1', columns='Country Name', values=0)
df3.to_csv('a.csv')



In [ ]:
df = pd.read_csv('a.csv')

df["level_1"] = df["level_1"].str[:4]
df['SSA'] = df['Sub-Saharan Africa']/1000000
df['UKR'] = df['Ukraine']/1000000
df2 = df.drop('Sub-Saharan Africa', axis =1)
df3 = df2.drop('Ukraine', axis =1)
df3.rename(columns={'level_1': 'Year'}, inplace=True)
#df4 = df3.set_index('Year')
#df4.to_csv('SSA_UKR3.csv')

#df5 = df4[df4['Year'] >= 2010]
#df4 = df3[df3['Year'] >= 2010]
#f4 = df3.drop(df3[df3['Year'] < 2010].index)
#df3['Year'].dtypes
#df3['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df3

,Year,SSA,UKR
0,1997,20825.939896,395.029999
1,1998,20429.819969,683.489990
2,1999,18663.859930,845.320007
3,2000,19577.660112,822.750000
4,2001,22068.159970,774.380005
5,2002,28228.789972,692.859985
6,2003,32075.380160,436.730011
7,2004,30933.859995,456.910004
8,2005,37122.879690,472.489990
9,2006,45192.369934,546.700012
